In [58]:
import numpy as np
import igl
import meshplot as mp
from scipy.spatial.transform import Rotation
import ipywidgets as iw
import time
from scipy.sparse import csc_matrix,diags,lil_matrix,vstack
from scipy.sparse.linalg import spsolve,inv
from scipy.linalg import det
from scipy.spatial import Delaunay
from IPython.display import display, HTML,IFrame
import scipy.sparse as sp

interactive = True
use_cholesky = False
filename = 'woody-hi'
html=False
triangulation_methods=0 # 0 for scipy, and 1 for triangle and using barycentric coordinates to sample interior points

In [59]:
if interactive:
    html=False
if html:
    mp.website()
file_counter = 0
def htmlit(plot,method=1):
    if method==1:
        global file_counter
        unique_filename = f"./htmls/plot_{filename}{file_counter}.html"
        file_counter += 1
        plot.save(unique_filename)
        return unique_filename
    else:
        html = plot.to_html(imports=True, html_frame=False)
        return HTML(html)

In [60]:
v, f = igl.read_triangle_mesh(f'data/{filename}.off')
cage_verts=np.load("data/woody-hi.cage.npy")
v -= v.min(axis=0)
v /= v.max()

In [61]:
print(cage_verts)
print(v.shape[0])
tri_verts=np.vstack((cage_verts,v))[:,:2]
temp=Delaunay(tri_verts)
tri_verts,tri_faces=temp.points,temp.simplices
padding=np.zeros((tri_verts.shape[0],1))
tri_verts=np.hstack((tri_verts,padding))
p=mp.plot(tri_verts,tri_faces,shading={"wireframe":True})

[[ 0.2   0.    0.  ]
 [ 0.28 -0.04  0.  ]
 [ 0.39 -0.01  0.  ]
 [ 0.44  0.21  0.  ]
 [ 0.52  0.    0.  ]
 [ 0.62 -0.03  0.  ]
 [ 0.74  0.01  0.  ]
 [ 0.75  0.16  0.  ]
 [ 0.65  0.49  0.  ]
 [ 0.89  0.49  0.  ]
 [ 0.89  0.72  0.  ]
 [ 0.62  0.73  0.  ]
 [ 0.58  0.93  0.  ]
 [ 0.41  1.1   0.  ]
 [ 0.22  0.94  0.  ]
 [ 0.19  0.81  0.  ]
 [ 0.26  0.73  0.  ]
 [-0.03  0.73  0.  ]
 [-0.03  0.48  0.  ]
 [ 0.23  0.48  0.  ]
 [ 0.13  0.12  0.  ]]
2642


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.4299999…

In [62]:
handle_vertex_positions = cage_verts.copy()
pos_f_saver = np.zeros((cage_verts.shape[0], 3))
def pos_f(s,x,y,z):
    #slices = (labels==s)
    #r = Rotation.from_euler('xyz', [α, β, γ], degrees=True)
    v_slice = cage_verts[s] + np.array([x,y,z])
    handle_vertex_positions[s] = v_slice
    pos_f_saver[s - 1] = [x,y,z]
    t0 = time.time()
    v_deformed = pos_f.deformer(handle_vertex_positions)
    #v_deformed = add_detail(v_deformed)
    print(v_deformed.shape)
    global lines_obj
    p.update_object(vertices = v_deformed,colors=np.array(weights[:,s]))
    p.remove_object(lines_obj)

    lines_obj=p.add_lines(handle_vertex_positions,np.vstack((handle_vertex_positions[1:],handle_vertex_positions[0])))
    #p.add_lines(handle_vertex_positions)
    t1 = time.time()
    print('FPS', 1/(t1 - t0))
#pos_f.deformer = lambda x:x

In [63]:
def widgets_wrapper():
    segment_widget = iw.Dropdown(options=np.arange(cage_verts.shape[0]))
    translate_widget = {i:iw.FloatSlider(min=-1, max=1, value=0) 
                        for i in 'xyz'}
    #rotate_widget = {a:iw.FloatSlider(min=-90, max=90, value=0, step=1) for a in 'αβγ'}

    def update_seg(*args):
        (translate_widget['x'].value,translate_widget['y'].value,
        translate_widget['z'].value) = pos_f_saver[segment_widget.value]
    segment_widget.observe(update_seg, 'value')
    widgets_dict = dict(s=segment_widget)
    widgets_dict.update(translate_widget)
    #widgets_dict.update(rotate_widget)
    return widgets_dict

In [64]:
def find_barycentric_triangle(obj_verts,cage_verts,cage_faces):
    index=np.zeros(obj_verts.shape[0])
    weight=np.zeros((obj_verts.shape[0],3))
    for i in range(obj_verts.shape[0]):
        for j in range(cage_faces.shape[0]):
            res=igl.barycentric_coordinates_tri(obj_verts[i],cage_verts[cage_faces[j,0]],cage_verts[cage_faces[j,1]],cage_verts[cage_faces[j,2]])
            if res[0]>=0 and res[1]>=0 and res[2]>=0:
                index[i]=j
                weight[i]=res
                break
    return index,weight
    
def cagesolver_2d(tri_verts,tri_faces,cage_verts):
    #index,weight=find_barycentric_triangle(verts,triangulated_cage,cage_f)
    mass_mat=igl.massmatrix(tri_verts,tri_faces,igl.MASSMATRIX_TYPE_BARYCENTRIC)
    M=mass_mat
    cot_mat=igl.cotmatrix(tri_verts,tri_faces)
    L=inv(M)@cot_mat
    '''
    d=np.ones((cage.shape[0]))
    d=diags(d)
    consA=lil_matrix((cage.shape[0],tri_verts.shape[0]))
    for i in range(cage.shape[0]):
        consA[:,:cage.shape[0]]=d
        L_prime=L.copy().tolil()
        L_prime[:cage.shape[0],:]=consA
        #consA=consA.tocsc()
        
        consA=np.zeros((cage_verts.shape[0],tri_verts.shape[0]))
        for j in range(cage.shape[0]):
            consA[j,j]=1
        
        
        consb=np.zeros((tri_verts.shape[0],1))
        consb[i]=1

        result[:,i]=spsolve(L_prime,consb)[cage_verts.shape[0]:]
    '''
    Lff=L[cage_verts.shape[0]:,:][:,cage_verts.shape[0]:]
    Lfc=L[cage_verts.shape[0]:,:][:,:cage_verts.shape[0]]
    xc=np.ones(cage_verts.shape[0])
    xc=diags(xc)
    result=spsolve(Lff,-Lfc@xc)
    return result
    

def cage_deformer(cage):
    res=weights@cage
    return res
pos_f.deformer=cage_deformer

In [65]:
weights=cagesolver_2d(tri_verts,tri_faces,cage_verts).todense()


In [66]:
np.array(weights[:,0]).shape

(2642, 1)

In [67]:
## Widget UI
if interactive==True:
    p = mp.plot(v, f)
    lines_obj=p.add_lines(handle_vertex_positions,np.vstack((handle_vertex_positions[1:],handle_vertex_positions[0])))

    iw.interact(pos_f, **widgets_wrapper())

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.4306930…

interactive(children=(Dropdown(description='s', options=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15,…

In [56]:
from sksparse.cholmod import cholesky
factor = cholesky(Aff)
def Cholesky_deformer(target_pos):
    b=-Afc@target_pos[slicesconstraint]
    x = factor(b)
    new=target_pos.copy()
    new[slicesfree]=x
    return new

if use_cholesky:
    pos_f.deformer=Cholesky_deformer

NameError: name 'Aff' is not defined

In [57]:
def manual_pos_f(verts, labels, s, x, y, z, α, β, γ):
    slices = (labels==s)
    r = Rotation.from_euler('xyz', [α, β, γ], degrees=True)
    v_slice = v[slices] + np.array([[x,y,z]])
    center = v_slice.mean(axis=0)
    verts[slices] = r.apply(v_slice - center) + center
    v_smoothed = pos_f.deformer(verts) # B'
    v_deformed = add_detail(v_smoothed) # S'
    return v_smoothed,v_deformed
    
if interactive==False:
    c = np.load(f'data/{filename}.pos.npy')
    V=v
    #B,_=manual_pos_f(V,labels,0,0,0,0,0,0)
    B_prime=V
    for i in range(c.shape[0]):
        B_prime,S_prime=manual_pos_f(V,labels,c[i,0].astype(int),c[i,1],c[i,2],c[i,3],c[i,4],c[i,5],c[i,6])
    p1= mp.plot(B_prime, f, c=labels)
    p2= mp.plot(S_prime, f, c=labels)

In [ ]:
if html and interactive==False:
    html=htmlit(p1)
IFrame(src=html, width=700, height=600)

In [ ]:
if html and interactive==False:
    html=htmlit(p2)
IFrame(src=html, width=700, height=600)